In [28]:
import pandas as pd 
import numpy as np
from scipy.sparse import csr_matrix
from fuzzywuzzy import process

In [3]:
movies = "../data/ml-latest-small/movies.csv"
ratings = "../data/ml-latest-small/ratings.csv"

df_movies = pd.read_csv(movies, usecols = ["movieId", "title"], dtype={"movieId":"int32", "title":"str"})
df_ratings = pd.read_csv(ratings, usecols = ["userId", "movieId", "rating"], dtype={"userId":"int32", "movieId":"int32",  "rating":"float32"})

In [9]:
df_ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [36]:
#spare Matrix
#            users
#           [4,5,4] A
#movies     [3,3,4] B  Cos(A,B) => 0.95
#           [3,2,1]

movies_users = df_ratings.pivot(index="movieId", columns= "userId", values="rating").fillna(0)
movies_users.head()



mat_movies_users = csr_matrix(movies_users.values)




In [43]:
# Euclidean Distance
# Manhattan Distance
# Minkowski Distance
# Cosine Similarity
from sklearn.neighbors import NearestNeighbors
model_knn = NearestNeighbors(metric= "cosine", algorithm="brute", n_neighbors = 10)

In [44]:
model_knn.fit(mat_movies_users)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=10)

In [48]:
from sklearn.metrics import pairwise_distances

def recommender(movie_name, data, model, n_recommendations):
    # Använd extractOne för att hitta rätt movie_name
    idx = process.extractOne(movie_name, df_movies["title"])[2]
    print("Selected movie:", df_movies["title"][idx], "Index:", idx)
    
    # Använd getrow för att hämta data för det valda indexet
    selected_row = data.getrow(idx)
    
    # Träna modellen
    model.fit(data)
    
    # Beräkna avstånden till närmaste grannar
    distances, indices = model.kneighbors(selected_row, n_neighbors=n_recommendations)
    
    print("Searching for recommendations...")
    # Loopa igenom rekommendationerna och skriv ut titlarna med Euclidean-distans
    for i in range(len(indices[0])):
        if indices[0][i] != idx:  # Undvik att skriva ut samma film som input
            recommended_idx = indices[0][i]
            recommended_movie = df_movies["title"][recommended_idx]
            distance = pairwise_distances(selected_row, data.getrow(recommended_idx), metric='euclidean')
            print(f"Recommendation {i+1}: {recommended_movie}, Euclidean Distance: {distance[0][0]}")

# Använd funktionen med rätt parametrar
recommender("Toy Story", mat_movies_users, model_knn, 5)

Selected movie: Toy Story (1995) Index: 0
Searching for recommendations...
Recommendation 2: 'night Mother (1986), Euclidean Distance: 48.46906280517578
Recommendation 3: Jurassic Park (1993), Euclidean Distance: 55.04316329956055
Recommendation 4: Independence Day (a.k.a. ID4) (1996), Euclidean Distance: 51.686553955078125
Recommendation 5: Star Wars: Episode IV - A New Hope (1977), Euclidean Distance: 60.448326110839844


In [46]:
# Recommender (movie_name) => List of movies recommended

def recommender (movie_name, data, model, n_recommendations):
    model.fit(data)
    idx=process.extractOne(movie_name, df_movies["title"])[2]
    print("movie Selected:" ,df_movies["title"][idx], "Index:",idx)
    print("searching for recommendations.....")
    distances, indices=model.kneighbors(data.getrow(idx), n_neighbors=n_recommendations)
    for i in indices:
        print(df_movies["title"][i].where(i!=idx))

    
recommender("Psycho", mat_movies_users, model_knn,10)

movie Selected: Psycho (1960) Index: 920
searching for recommendations.....
920                                                   NaN
862                               Private Benjamin (1980)
963                                           Diva (1981)
819                           Fish Called Wanda, A (1988)
1430                        French Connection, The (1971)
908                          To Kill a Mockingbird (1962)
818                                        Bananas (1971)
2216                              Dirty Dozen, The (1967)
968                           Arsenic and Old Lace (1944)
910     Once Upon a Time in the West (C'era una volta ...
Name: title, dtype: object
